# Connected Components

The purpose of this assignment is to familiarize yourself with the handling of graph data structures. You will implement depth-first search for identifying the connected components of an undirected graph, implementing procedure Search as a subroutine along the way.

You will use the [NetworkX](https://networkx.github.io/) Python package to represent and manipulate graphs. You should first familiarize yourself with its functionality by going through the brief [tutorial](http://networkx.github.io/documentation/networkx-1.9.1/tutorial/index.html). For this homework, you may only use the basic undirected graph methods listed [here](http://networkx.github.io/documentation/networkx-1.9.1/reference/classes.graph.html).

As a use case, we will work with a dataset recording the interactions between characters in Homer's *Iliad*.

In [97]:
import networkx
import urllib2
homer = urllib2.urlopen('http://people.sc.fsu.edu/~jburkardt/datasets/sgb/homer.dat')

In [17]:
hasattr(homer, 'next')

True

In [2]:
lst = homer.readlines()

In [3]:
lst

['* File "homer.dat" from the Stanford GraphBase (C) 1992 Stanford University\n',
 '* $\\rm I\\Lambda IA\\Delta O\\Sigma$, by Homer\n',
 '* This file may be freely copied but please do not change it in any way!\n',
 '* (Checksum parameters 670,795252274)\n',
 'AA Aethra, Trojan lady in waiting\n',
 'AB Abarbarea, Trojan fountain nymph\n',
 'AC Achilles, angry warrior, swift-footed chief of Myrmidons from Phthia\n',
 'AD Automedon, charioteer of AC\n',
 'AE Aeneas, leader of Dardanians\n',
 'AF Aphrodite (Venus), daughter of ZE and DN, roots for Trojans\n',
 'AG Agamemnon, king of Argos and Mycenae, leader of Greek forces\n',
 'AH Andromache, wife of HT\n',
 'AI Anchises, father of AE\n',
 'AJ Great Ajax, king of Salamis\n',
 'AL Antilochus, son of NE\n',
 'AM Artemis (Cynthia/Diana), daughter of ZE and LE, roots for Trojans\n',
 'AN Antenor, aged councilor to PR\n',
 'AO Agenor, heir of AN, assists AE\n',
 'AP Apollo, son of ZE and LE, roots for Trojans\n',
 'AR Ares (Mars), son of ZE,

The format of the data is straightforward. After some comment lines (beginning with \*), the file lists a codename for each character (i.e., node of the graph), followed by a description. The file then lists the groups of characters that interact in each chapter, from which you will form the edges. For instance, the first line has the form:

```1:CH,AG,ME,GS;AP,CH;HE,AC;AC,AG,CA;HE,AT;AT,AC;AT,OG;NE,AG,AC;CS,OD```

This means that CH,AG,ME,GS interacted, so there are edges for all pairs of these nodes. Groups of characters that interacted are separated by semicolons. The lines start with chapter information of the form `1:` or `&:`, which can be ignored for this problem.

First implement a function to read in the nodes from the input file. You may implement any auxiliary functions as needed, and are encouraged to use small functions with specific purposes to keep your code readable. Any function you implement should be clearly commented.

In [58]:
def read_nodes(gfile):
    """Reads in the nodes of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top.
        
    Returns:
        A generator of the nodes in the graph, yielding a list of the form:
            ['CH', 'AG, 'ME', ...]
    """
    # TODO: check if generator or list
    nxt = gfile.next().strip()
    while nxt and nxt[0] == '*':
        nxt = gfile.next().strip()
    return_list = []
    while nxt:
        return_list.append(nxt)
        nxt = gfile.next().strip()
    return [each.split()[0] for each in return_list]
    ###if not nxt:
    ###    raise StopIteration # stops on first line with only whitespace
    
    ###yield nxt.split()[0]

In [60]:
print read_nodes(homer)

['AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'AG', 'AH', 'AI', 'AJ', 'AL', 'AM', 'AN', 'AO', 'AP', 'AR', 'AS', 'AT', 'AU', 'AX', 'AZ', 'BL', 'BO', 'BR', 'CA', 'CH', 'CI', 'CL', 'CM', 'CN', 'CR', 'CS', 'CT', 'DE', 'DI', 'DM', 'DN', 'DP', 'DT', 'EA', 'EB', 'EE', 'EF', 'EM', 'EN', 'EO', 'EP', 'ER', 'EU', 'FD', 'FY', 'GL', 'GR', 'GS', 'HA', 'HB', 'HC', 'HD', 'HE', 'HL', 'HM', 'HN', 'HO', 'HP', 'HR', 'HT', 'IA', 'ID', 'IR', 'LA', 'LE', 'LT', 'LY', 'MC', 'ME', 'MG', 'MO', 'MR', 'MT', 'MU', 'NE', 'NI', 'NO', 'NR', 'OC', 'OD', 'OG', 'OT', 'PA', 'PB', 'PC', 'PD', 'PE', 'PH', 'PL', 'PN', 'PO', 'PP', 'PR', 'PS', 'PT', 'PU', 'PX', 'RA', 'RH', 'RO', 'RU', 'SA', 'SE', 'SF', 'SI', 'SL', 'SP', 'ST', 'TA', 'TE', 'TH', 'TI', 'TL', 'TM', 'TR', 'TS', 'TT', 'TU', 'TY', 'WI', 'XA', 'XB', 'ZE', 'ZF', '01', '02', '03', '04', '05', '06', '07', '08', '09', '0A', '0B', '0C', '0D', '0E', '0F', '0G', '0H', '0I', '0J', '0K', '0L', '0M', '0N', '0O', '0P', '0Q', '0R', '0S', '0T', '0U', '0V', '0W', '0X', '0Y', '0Z', '10', '11

Next implement a function to read in the edges from the input file.

In [62]:
def read_edges(gfile):
    """Reads in the edges of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top 
            of the edges section.
            
    Returns:
        A generator of the edges in the graph, yielding a list of pairs of the form:
            [('CH', 'AG'), ('AG', 'ME'), ...]
    """
    # defining function within this function because of piazza instructions to not write outside
    def find_pairwise_edges(group_token):
        """
        # TODO: document
        """
        return_lst = []
        nodes = group_token.split(',')
        curr_node = nodes.pop()
        while nodes:
            for n in nodes:
                return_lst.append((curr_node, n))
            curr_node = nodes.pop()
        return return_lst
    to_return = []
    try:
        while True:
            line = gfile.next().strip()
            if line[0] == '*':
                # per assignment instructions, ignore input file lines that start with *
                continue
            line = line.split(':')[1]
            for grp in line.split(';'):
                for edge in find_pairwise_edges(grp):
                    to_return.append(edge)
    except StopIteration:
        # this condition is reached when file stream is empty
        pass
    return to_return

In [61]:
print read_edges(homer)

StopIteration: 

The following code should now correctly create the graph.

In [106]:
import networkx as nx
homer = urllib2.urlopen('http://people.sc.fsu.edu/~jburkardt/datasets/sgb/homer.dat') ### DELETE THIS
G = nx.Graph()
G.add_nodes_from(read_nodes(homer))
G.add_edges_from(read_edges(homer))

In [117]:
# DFS searching alphabetically, should be working
test_node = 'SF'
def DFS(graph, node, return_list, bool_mark=True):   
    if 'visited' in graph.nodes[node] and graph.nodes[node]['visited'] == bool_mark:
        return return_list
    # visiting node
    return_list.append(node)
    graph.nodes[node]['visited'] = bool_mark
    alphabetical_neighbors = sorted(G.adj[node].keys())
    for neighbor in alphabetical_neighbors:
        DFS(graph, neighbor, return_list, bool_mark=bool_mark)
    return return_list
DFS(G,'SF',[])
DFS(G,'SF',[], bool_mark = False)


['SF',
 'AP',
 '1K',
 '0P',
 '0N',
 '0O',
 '0Q',
 '0R',
 '0S',
 'AA',
 'AF',
 'AE',
 '01',
 '02',
 '09',
 '0A',
 '9K',
 '7R',
 '7S',
 '7T',
 '9W',
 '9V',
 '9X',
 'AL',
 '06',
 '3W',
 '3X',
 '3Y',
 '3Z',
 '40',
 '93',
 '44',
 '45',
 '46',
 '92',
 '94',
 '22',
 '41',
 '42',
 '43',
 '95',
 '6Y',
 'AC',
 '0M',
 '0L',
 '2I',
 'AH',
 '2J',
 'HT',
 '07',
 '08',
 '0F',
 '0G',
 '0H',
 '0I',
 '0J',
 '0K',
 '3V',
 'AJ',
 '0W',
 '1J',
 '1U',
 '34',
 '35',
 'AG',
 '03',
 '04',
 '05',
 'DI',
 '0Y',
 '0Z',
 '10',
 'HP',
 '90',
 'TH',
 '5D',
 'PE',
 '5L',
 '5N',
 '5O',
 '5E',
 '5F',
 '5K',
 '5G',
 '5H',
 '5I',
 '5J',
 'HM',
 'AR',
 '2C',
 '2B',
 '2A',
 'BL',
 '29',
 '83',
 'ZE',
 '1F',
 '1G',
 'OG',
 '87',
 'DT',
 'SE',
 'AT',
 '13',
 'MR',
 '4G',
 '0C',
 '1I',
 'ME',
 '12',
 'AM',
 '2E',
 '2D',
 'GL',
 '5C',
 '7Z',
 'AO',
 '31',
 'PD',
 '2W',
 '4H',
 '4I',
 '4J',
 '4K',
 '4L',
 'PS',
 '36',
 '3T',
 '4C',
 'AI',
 'ID',
 '11',
 '4B',
 '4A',
 'HC',
 '14',
 '15',
 'AN',
 'CL',
 'HL',
 'IR',
 'BO',
 'EU',

In [105]:
G.nodes['AP']

{'visited': True}

Next implement procedure Search. The function takes in a graph and a root node, and returns a list of the nodes visited during the search. The nodes should appear in the order in which they were *first visited*. The neighbors of a node should be processed in *alphabetical order*, where numbers come before letters. This will ensure that the output of your function is uniquely defined, given any input node.

In [122]:
def Search(graph, root):
    """Runs depth-first search through a graph, starting at a given root. Neighboring
    nodes are processed in alphabetical order.
    
    Args:
        graph: the given graph, with nodes encoded as strings.
        root: the node from which to start the search.
        
    Returns:
        A list of nodes in the order in which they were first visited.
    """
    def DFS(graph, node, return_list, bool_mark=True):
        """
        TODO: comment
        """
        if 'visited' in graph.nodes[node] and graph.nodes[node]['visited'] == bool_mark:
            return return_list
        # visiting node
        return_list.append(node)
        graph.nodes[node]['visited'] = bool_mark
        alphabetical_neighbors = sorted(G.adj[node].keys())
        for neighbor in alphabetical_neighbors:
            DFS(graph, neighbor, return_list, bool_mark=bool_mark)
        return return_list
    # DFS, visiting all the nodes and returning visit node
    to_return = DFS(graph, root, []) 
    # calling again with bool_mark = False, to undo all the 'visited' markings for subsequent calls
    DFS(graph, root, [],bool_mark=False)
    return to_return

We will check the correctness of your code by verifying that it correctly computes the DFS tree starting at Ulysses (node `OD`).

In [124]:
ulysses = Search(G, 'OD')

In [125]:
ulysses

['OD',
 '0X',
 '1M',
 '1N',
 '1O',
 '1P',
 '1Q',
 '1R',
 '1S',
 '23',
 '2X',
 'DI',
 '03',
 '04',
 '05',
 'AG',
 '0D',
 '0E',
 '1H',
 '26',
 '32',
 '33',
 '34',
 '35',
 'AC',
 '0M',
 '0L',
 '2I',
 'AH',
 '2J',
 'HT',
 '02',
 '01',
 'AE',
 '0F',
 '07',
 '08',
 '0G',
 '0H',
 '0I',
 '0J',
 '0K',
 '3V',
 'AJ',
 '0W',
 '1J',
 '1U',
 '3N',
 '3O',
 '3P',
 '3Q',
 '3R',
 '47',
 '4P',
 '4V',
 '4Y',
 '5T',
 '6M',
 '6N',
 '8W',
 '8T',
 '8U',
 '8V',
 '8X',
 'NE',
 '5A',
 '57',
 'PO',
 '1K',
 '0P',
 '0N',
 '0O',
 '0Q',
 '0R',
 '0S',
 'AA',
 'AF',
 'AI',
 '3T',
 '4C',
 'AO',
 '31',
 'PD',
 '2W',
 '4H',
 '4I',
 '4J',
 '4K',
 '4L',
 'PS',
 '36',
 '4G',
 '0C',
 '1I',
 'AL',
 '06',
 '3W',
 '3X',
 '3Y',
 '3Z',
 '40',
 '93',
 '44',
 '45',
 '46',
 '92',
 '94',
 '22',
 '41',
 '42',
 '43',
 '95',
 '6Y',
 '96',
 '98',
 '9A',
 '9B',
 '9C',
 'AX',
 '4N',
 '6R',
 'MT',
 '6I',
 '6J',
 '6O',
 '6Q',
 'TU',
 '24',
 '2N',
 '2O',
 '2P',
 '2Q',
 '2R',
 '2S',
 '2T',
 '2U',
 '2V',
 '48',
 '49',
 'PR',
 '0T',
 '0U',
 '4A',

Next implement DFS to find the connected components of the character graph. When choosing roots for your components, always pick the *smallest unvisited node* according to alphabetical ordering. Combined with your Search routine, this will ensure that the output is again uniquely defined.

In [7]:
def connected_components(graph):
    """Computes the connected components of the given graph.
    
    Args: 
        graph: the given graph, with nodes encoded as strings.
        
    Returns:
        The connected components of the graph. Components are listed in
        alphabetical order of their root nodes.
    """
    # TODO: implement function
    pass

We will check correctness of your code by verifying that your output list is identical to our solution.

In [ ]:
character_interactions = connected_components(G)

As a preliminary check, you should find that the following statements are all true.

In [ ]:
component_sizes = [len(c) for c in character_interactions]
print "There are 12 connected components in the Iliad:", len(component_sizes) == 12
print "The giant component has size 542:", max(component_sizes) == 542
print "There are 5 isolated characters:", len([c for c in component_sizes if c == 1]) == 5